<a href="https://colab.research.google.com/github/antonis00/EKPA/blob/main/Eye_Movements_Initial_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Imports

import os
import re
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from google.colab import userdata

In [1]:
### Remove prefix from file names

# Directory where the original files are located
source_directory = r'C:\Users\apatergianakis\Desktop\Files\Personal\PhD\DataSets\Eye Movements\data'

# Directory where you want to save the new files
destination_directory = r'C:\Users\apatergianakis\Desktop\Files\Personal\PhD\DataSets\Eye Movements\data\renamed'

# Ensure the destination directory exists
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

# List all files in the source directory
file_list = os.listdir(source_directory)

# Loop through all files in the list
for file_name in file_list:
    # Check if the file is a CSV by extension and is not a directory
    if file_name.lower().endswith('.csv') and os.path.isfile(os.path.join(source_directory, file_name)):
        # Remove the 'S_' and the first digit following it
        new_name = re.sub(r'S_\d', '', file_name)
        # The complete path for the new file
        new_file_path = os.path.join(destination_directory, new_name)
        # The complete path for the old file
        old_file_path = os.path.join(source_directory, file_name)
        # Copy the file to the new name
        shutil.copy2(old_file_path, new_file_path)

SyntaxError: invalid syntax (<ipython-input-1-4ffdac8e4dcf>, line 1)

In [ ]:
### Add new column with the label
# Directory with the original CSV files
source_directory = r'C:\Users\apatergianakis\Desktop\Files\Personal\PhD\DataSets\Eye Movements\data\renamed'

# Directory where the new labeled files will be stored
destination_directory = os.path.join(source_directory, 'labeled')

# Ensure the destination directory exists
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

# Iterate over all files in the source directory
for filename in os.listdir(source_directory):
    if filename.endswith('.csv'):
        # Extract the participant ID from the filename
        participant_id = re.match(r'^(\d+)_', filename).group(1)

        # Construct the full file paths
        src_file_path = os.path.join(source_directory, filename)
        dest_file_path = os.path.join(destination_directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(src_file_path)

        # Add the participant_id column
        df['participant_id'] = participant_id

        # Save the modified DataFrame to the new location
        df.to_csv(dest_file_path, index=False)

In [ ]:
### Copy vid files only to a different folder

# Define the source and destination directories
source_directory = r'C:\Users\apatergianakis\Desktop\Files\Personal\PhD\DataSets\Eye Movements\data\renamed\labeled'
destination_directory = r'C:\Users\apatergianakis\Desktop\Files\Personal\PhD\DataSets\Eye Movements\data\renamed\labeled\vid'

# Create the destination directory if it does not exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

# Copy each CSV file that contains 'VID' in its name to the new subfolder
for filename in os.listdir(source_directory):
    if filename.endswith('.csv') and 'VID' in filename:
        # Construct full file paths
        src_file_path = os.path.join(source_directory, filename)
        dest_file_path = os.path.join(destination_directory, filename)

        # Copy the file
        shutil.copy2(src_file_path, dest_file_path)


In [ ]:
### Merge csv files to a single file

# Directory with the CSV files to merge
source_directory = r'C:\Users\apatergianakis\Desktop\Files\Personal\PhD\DataSets\Eye Movements\data\renamed\labeled\vid'

# Directory where the merged file will be stored
destination_directory = os.path.join(source_directory, 'one')

# Ensure the destination directory exists
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

# List to hold data from each CSV file
data_frames = []

# Iterate over all files in the source directory
for filename in os.listdir(source_directory):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(source_directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Append the DataFrame to the list
        data_frames.append(df)

# Concatenate all the data frames in the list
merged_df = pd.concat(data_frames, ignore_index=True)

# Path for the merged file
merged_file_path = os.path.join(destination_directory, 'merged.csv')

# Save the merged DataFrame to CSV
merged_df.to_csv(merged_file_path, index=False)


In [ ]:
## Count the participants from the merged file

# Path to the merged CSV file
merged_file_path = r'C:\Users\apatergianakis\Desktop\Files\Personal\PhD\DataSets\Eye Movements\data\renamed\labeled\vid\one\merged.csv'

# Check if the merged file exists
if os.path.exists(merged_file_path):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(merged_file_path)

    # Count the number of unique participant IDs
    unique_participants = df['participant_id'].nunique()

    print(f'There are {unique_participants} unique participants.')
else:
    print(f'The file {merged_file_path} does not exist.')


In [ ]:
# Load the dataset
df = pd.read_csv('/content/merged.csv')
# userdata.get('secretName') use secret

In [ ]:
# Assuming the DataFrame 'df' is already loaded with the eye-tracking data
features = ['x', 'y', 'lx', 'ly', 'rx', 'ry', 'clx', 'cly', 'clz', 'crx', 'cry', 'crz']
target = 'participant_id'

In [ ]:
# Separate features and target
X = df[features]
y = df[target]

In [ ]:
# Convert labels to categorical one-hot encoding
num_classes = 407  # You have 407 participants
y_one_hot = to_categorical(y, num_classes=num_classes)

In [ ]:
# Normalize feature values
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_one_hot, test_size=0.2, random_state=42)

In [ ]:
# Build a simple neural network model
model = Sequential()
model.add(Dense(128, input_dim=len(features), activation='relu'))
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))  # Output layer for 407 participants

In [ ]:
# Plot the depp learning model to an image
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Display the image of the model
from IPython.display import Image
Image(filename='model.png')

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',  # Use categorical_crossentropy for multi-class problems
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)

In [ ]:
# Evaluate the model
evaluation = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {evaluation[0]}')
print(f'Test Accuracy: {evaluation[1]}')

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

In [ ]:
# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

In [ ]:
plt.tight_layout()
plt.show()